In [1]:
!pip install librosa torchaudio openai-whisper tqdm
!apt-get install ffmpeg
!pip install noisereduce scipy matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 35.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.1 MB

In [3]:
import os
import torch
import librosa
import librosa.display
import numpy as np
import torchaudio
import torchaudio.transforms as T
import whisper
import matplotlib.pyplot as plt
import noisereduce as nr
import scipy.signal
from tqdm import tqdm
from glob import glob
from google.colab import drive


In [4]:
drive.mount('/content/drive')  # Mount Google Drive

Mounted at /content/drive


In [5]:
# Define paths
AUDIO_DIR = '/content/drive/MyDrive/captchaDatabase/captchas/audio'
OUTPUT_FILE = '/content/drive/MyDrive/captcha_transcriptions.txt'

In [6]:
# Load model
model = whisper.load_model("medium").to("cuda")

100%|█████████████████████████████████████| 1.42G/1.42G [00:19<00:00, 78.7MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [7]:
def load_audio(file_path, sr=16000):
    """Load an audio file."""
    audio, sample_rate = librosa.load(file_path, sr=sr)
    return audio, sample_rate

In [8]:
def noise_reduction(audio, sr):
    """Reduce noise from the audio signal."""
    reduced_noise_audio = nr.reduce_noise(y=audio, sr=sr)
    return reduced_noise_audio

In [9]:
def normalize_audio(audio):
    """Normalize audio signal to range [-1,1]."""
    return librosa.util.normalize(audio)

In [10]:
def extract_mfccs(audio, sr, n_mfcc=13):
    """Extract MFCC features from the audio signal."""
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return mfccs

In [11]:
def extract_spectrogram(audio, sr, n_fft=2048, hop_length=512):
    """Extract spectrogram from audio signal."""
    spectrogram = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)
    spectrogram_db = librosa.amplitude_to_db(abs(spectrogram))
    return spectrogram_db

In [12]:
def process_audio_files(audio_dir, sample_size=500):
    """Process a limited number of audio files in the dataset."""
    audio_files = sorted(glob(os.path.join(audio_dir, '*.wav')))[:sample_size]  # Get first 500 files
    features = {}

    for file in tqdm(audio_files, desc="Processing audio files"):
        file_name = os.path.basename(file)

        # Load and preprocess audio
        audio, sr = load_audio(file)
        audio = noise_reduction(audio, sr)
        audio = normalize_audio(audio)

        # Extract features
        mfccs = extract_mfccs(audio, sr)
        spectrogram = extract_spectrogram(audio, sr)

        features[file_name] = {'mfccs': mfccs, 'spectrogram': spectrogram}

    return features


In [13]:
def transcribe_audio(audio_dir, output_file, sample_size=500):
    """Transcribe a limited number of audio CAPTCHA files using Whisper and save results."""
    model = whisper.load_model("base")
    audio_files = sorted(glob(os.path.join(audio_dir, '*.wav')))[:sample_size]  # Get first 500 files

    with open(output_file, 'w') as f:
        for file in tqdm(audio_files, desc="Transcribing audio files"):
            file_name = os.path.basename(file)
            result = model.transcribe(file)
            transcript = result['text']
            f.write(f"{file_name}: {transcript}\n")

# Run preprocessing, feature extraction, and transcription for first 500 samples
features = process_audio_files(AUDIO_DIR, sample_size=500)
transcribe_audio(AUDIO_DIR, OUTPUT_FILE, sample_size=500)
print("Feature extraction and transcription completed!")

Processing audio files: 100%|██████████| 500/500 [07:32<00:00,  1.10it/s]
100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 31.0MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded

Feature extraction and transcription completed!
